In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [74]:
df = pd.read_csv('SZFZ_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df.drop('DateConfirm',axis=1,inplace=True)
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[df['Class']=='SZFZ','Class'] = 0
df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

p_total = 1118
c_total = 18

PERIODS = [2.5,0.5,1,1.3,2.5,1,1,1.5,1.3,1.3,1.1,3,0.5,1.3]

TYPES = ['I','period']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [p_total/c_total,] * (len(CLASSES)-1)
POPULATION = np.append(POPULATION,[p_total,])

assert(len(PERIODS)==len(CLASSES) and len(POPULATION)==len(CLASSES))

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,'I'] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            l += 1

for t in ['I',]:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)
            
for (c,p,r) in zip(CLASSES,POPULATION,PERIODS):
    org_df.loc[:,('I',c)] /= p
    org_df.loc[date_index[0],('period',c)] = r

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

52.0           51.0             48.0           46.0         \
              I period       I period         I period       I period   
2013-12  0.0000    2.5  0.0000    0.5  0.000000      1  0.0000    1.3   
2014-01  0.0000    NaN  0.0000    NaN  0.000000    NaN  0.0000    NaN   
2014-02  0.0000    NaN  0.0000    NaN  0.000000    NaN  0.0000    NaN   
2014-03  0.0000    NaN  0.0000    NaN  0.000000    NaN  0.0000    NaN   
2014-04  0.0000    NaN  0.0000    NaN  0.016100    NaN  0.0000    NaN   
2014-05  0.0161    NaN  0.0161    NaN  0.013417    NaN  0.0322    NaN   

             45.0         ...        40.0           37.0           35.0  \
                I period  ...           I period       I period       I   
2013-12  0.000000    2.5  ...    0.000000    1.3  0.0000    1.1  0.0000   
2014-01  0.000000    NaN  ...    0.000000    NaN  0.0000    NaN  0.0000   
2014-02  0.000000    NaN  ...    0.000000    NaN  0.0000    NaN  0.0000   
2014-03  0.000000    NaN  ...    0.000000    NaN  0.0000    NaN  0.0000   
2014-04  0.016100    NaN  ...    0.032200    NaN  0.0000    NaN  0.0000   
2014-05  0.029517    NaN  ...    0.075134    NaN  0.0161    NaN  0.0161   

                    0.0              1.0          
        period         I period         I period  
2013-12      3  0.000000    0.5  0.000894    1.3  
2014-01    NaN  0.000000    NaN  0.000745    NaN  
2014-02    NaN  0.048301    NaN  0.003305    NaN  
2014-03    NaN  0.072451    NaN  0.005437    NaN  
2014-04    NaN  0.173077    NaN  0.025103    NaN  
2014-05    NaN  0.144231    NaN  0.053120    NaN  

[6 rows x 28 columns]

In [69]:
org_df.to_csv('SZFZ_ver3.csv')

In [83]:
df = pd.read_csv('SYZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])

df.drop('DateConfirm',axis=1,inplace=True)
df.drop(94,inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

df.loc[df['Class'].isnull(),'Class'] = 0

date_start = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

p_total = 2760
c_total = 30

PERIODS = [0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.9,0.4,0.6,0.6,0.6,0.6,0.6]

TYPES = ['I','period']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [p_total/c_total,] * (len(CLASSES)-1)
POPULATION = np.append(POPULATION,[p_total,])

assert(len(PERIODS)==len(CLASSES) and len(POPULATION)==len(CLASSES))

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,'I'] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            l += 1

for t in ['I',]:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)
            
for (c,p,r) in zip(CLASSES,POPULATION,PERIODS):
    org_df.loc[:,('I',c)] /= p
    org_df.loc[date_index[0],('period',c)] = r

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

104.0            102.0            101.0            100.0         \
                I period         I period         I period         I period   
2013-03  0.000000    0.6  0.000000    0.6  0.000000    0.6  0.000000    0.6   
2013-04  0.000000    NaN  0.000000    NaN  0.000000    NaN  0.000000    NaN   
2013-05  0.000000    NaN  0.000000    NaN  0.000000    NaN  0.000000    NaN   
2013-06  0.000000    NaN  0.000000    NaN  0.000000    NaN  0.000000    NaN   
2013-07  0.000000    NaN  0.000000    NaN  0.010870    NaN  0.000000    NaN   
2013-08  0.021739    NaN  0.000000    NaN  0.009058    NaN  0.021739    NaN   
2013-09  0.018116    NaN  0.000000    NaN  0.007548    NaN  0.018116    NaN   
2013-10  0.015097    NaN  0.010870    NaN  0.017160    NaN  0.015097    NaN   
2013-11  0.012581    NaN  0.009058    NaN  0.014300    NaN  0.012581    NaN   

            99.0          ...       94.0             93.0             92.0   \
                I period  ...           I period         I period         I   
2013-03  0.000000    0.6  ...    0.000000    0.6  0.010870    0.6  0.000000   
2013-04  0.000000    NaN  ...    0.000000    NaN  0.009058    NaN  0.010870   
2013-05  0.000000    NaN  ...    0.000000    NaN  0.007548    NaN  0.009058   
2013-06  0.021739    NaN  ...    0.000000    NaN  0.082377    NaN  0.018418   
2013-07  0.039855    NaN  ...    0.000000    NaN  0.101256    NaN  0.015348   
2013-08  0.033213    NaN  ...    0.000000    NaN  0.193076    NaN  0.012790   
2013-09  0.060286    NaN  ...    0.010870    NaN  0.258723    NaN  0.021528   
2013-10  0.115456    NaN  ...    0.009058    NaN  0.248211    NaN  0.028810   
2013-11  0.107083    NaN  ...    0.007548    NaN  0.206842    NaN  0.024008   

                   0.0              1.0           
        period         I period         I period  
2013-03    0.6  0.000000    0.6  0.000362    0.6  
2013-04    NaN  0.000000    NaN  0.000664    NaN  
2013-05    NaN  0.000000    NaN  0.000916    NaN  
2013-06    NaN  0.000000    NaN  0.004386    NaN  
2013-07    NaN  0.000000    NaN  0.006916    NaN  
2013-08    NaN  0.010870    NaN  0.012285    NaN  
2013-09    NaN  0.009058    NaN  0.016397    NaN  
2013-10    NaN  0.007548    NaN  0.019824    NaN  
2013-11    NaN  0.017160    NaN  0.017244    NaN  

[9 rows x 28 columns]

In [84]:
org_df.to_csv('SYZX_ver3.csv')

In [79]:
df = pd.read_csv('MBZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])

df.drop('DateConfirm',axis=1,inplace=True)
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])

df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

PERIODS = [5,5,5]

TYPES = ['I','period']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [51,68,303]

assert(len(PERIODS)==len(CLASSES) and len(POPULATION)==len(CLASSES))

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,'I'] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            l += 1

for t in ['I',]:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)
            
for (c,p,r) in zip(CLASSES,POPULATION,PERIODS):
    org_df.loc[:,('I',c)] /= p
    org_df.loc[date_index[0],('period',c)] = r

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

8.0              7.0              1.0       
                I period         I period         I period
2014-03  0.000000      5  0.000000      5  0.000000      5
2014-04  0.019608    NaN  0.000000    NaN  0.003300    NaN
2014-05  0.016340    NaN  0.000000    NaN  0.002750    NaN
2014-06  0.033224    NaN  0.000000    NaN  0.005592    NaN
2014-07  0.047295    NaN  0.000000    NaN  0.007961    NaN
2014-08  0.059020    NaN  0.000000    NaN  0.009934    NaN
2014-09  0.147223    NaN  0.014706    NaN  0.028080    NaN

In [80]:
org_df.to_csv('MBZX_ver3.csv')

In [81]:
df = pd.read_csv('QJYS_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])

df.drop('DateConfirm',axis=1,inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

p_total = 1958
c_total = 26

PERIODS = [6,6,6]

TYPES = ['I','period']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [p_total/c_total,] * (len(CLASSES)-1)
POPULATION = np.append(POPULATION,[p_total,])

assert(len(PERIODS)==len(CLASSES) and len(POPULATION)==len(CLASSES))

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,'I'] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            l += 1

for t in ['I',]:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)
            
for (c,p,r) in zip(CLASSES,POPULATION,PERIODS):
    org_df.loc[:,('I',c)] /= p
    org_df.loc[date_index[0],('period',c)] = r

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

230.0            223.0            1.0         
                I period         I period         I period
2017-06  0.000000      6  0.013279      6  0.000511      6
2017-07  0.000000    NaN  0.011066    NaN  0.000426    NaN
2017-08  0.000000    NaN  0.009221    NaN  0.000355    NaN
2017-09  0.000000    NaN  0.007685    NaN  0.000296    NaN
2017-10  0.000000    NaN  0.019683    NaN  0.000757    NaN
2017-11  0.000000    NaN  0.029681    NaN  0.001142    NaN
2017-12  0.000000    NaN  0.064571    NaN  0.002483    NaN
2018-01  0.013279    NaN  0.053809    NaN  0.002580    NaN

In [82]:
org_df.to_csv('QJYS_ver3.csv')

In [42]:
df = pd.read_csv('SYZX_monthly_ver2.csv',header=[0,1],index_col=0)

In [ ]:
def organize3(name,population,periods):
    df = pd.read_csv(name+'_monthly_ver2.csv',header=[0,1],index_col=0)
    df.drop(['I_new','A(not tp)','tp','I/(I+A)','I/population'],axis=1,level=1)
    classes = df.columns.levels[0].values
    

In [47]:
df.columns.levels[0].values

array(['0.0', '1.0', '100.0', '101.0', '102.0', '104.0', '92.0', '93.0',
       '94.0', '95.0', '96.0', '97.0', '98.0', '99.0'], dtype=object)

In [30]:
df.drop('I_new',axis=1,level=1)

104.0                                              102.0  \
                I A(not tp)   tp    A I/(I+A) I/population         I   
2013-03  0.000000       3.0  0.0  3.0     0.4     0.015552  0.000000   
2013-04  0.000000       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-05  0.000000       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-06  0.000000       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-07  0.000000       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-08  2.000000       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-09  1.666667       NaN  NaN  NaN     NaN          NaN  0.000000   
2013-10  1.388889       NaN  NaN  NaN     NaN          NaN  1.000000   
2013-11  1.157407       NaN  NaN  NaN     NaN          NaN  0.833333   

                                ...       0.0                              \
        A(not tp)   tp    A     ...        tp    A   I/(I+A) I/population   
2013-03       1.0  0.0  1.0     ...       0.0  1.0  0.666667     0.015552   
2013-04       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-05       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-06       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-07       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-08       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-09       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-10       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   
2013-11       NaN  NaN  NaN     ...       NaN  NaN       NaN          NaN   

               1.0                                            
                 I A(not tp)   tp     A I/(I+A) I/population  
2013-03   1.000000      17.0  2.0  19.0     0.8     0.019699  
2013-04   1.833333       NaN  NaN   NaN     NaN          NaN  
2013-05   2.527778       NaN  NaN   NaN     NaN          NaN  
2013-06  12.106481       NaN  NaN   NaN     NaN          NaN  
2013-07  19.088735       NaN  NaN   NaN     NaN          NaN  
2013-08  33.907279       NaN  NaN   NaN     NaN          NaN  
2013-09  45.256066       NaN  NaN   NaN     NaN          NaN  
2013-10  54.713388       NaN  NaN   NaN     NaN          NaN  
2013-11  47.594490       NaN  NaN   NaN     NaN          NaN  

[9 rows x 84 columns]

In [41]:
df.drop()

104.0                                                    102.0  \
        I_new         I A(not tp)   tp    A I/(I+A) I/population I_new   
2013-03     0  0.000000       3.0  0.0  3.0     0.4     0.015552     0   
2013-04     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-05     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-06     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-07     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-08     2  2.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-09     0  1.666667       NaN  NaN  NaN     NaN          NaN     0   
2013-10     0  1.388889       NaN  NaN  NaN     NaN          NaN     1   
2013-11     0  1.157407       NaN  NaN  NaN     NaN          NaN     0   

                                ...       0.0                          1.0  \
                I A(not tp)     ...         A   I/(I+A) I/population I_new   
2013-03  0.000000       1.0     ...       1.0  0.666667     0.015552     1   
2013-04  0.000000       NaN     ...       NaN       NaN          NaN     1   
2013-05  0.000000       NaN     ...       NaN       NaN          NaN     1   
2013-06  0.000000       NaN     ...       NaN       NaN          NaN    10   
2013-07  0.000000       NaN     ...       NaN       NaN          NaN     9   
2013-08  0.000000       NaN     ...       NaN       NaN          NaN    18   
2013-09  0.000000       NaN     ...       NaN       NaN          NaN    17   
2013-10  1.000000       NaN     ...       NaN       NaN          NaN    17   
2013-11  0.833333       NaN     ...       NaN       NaN          NaN     2   

                                                              
                 I A(not tp)   tp     A I/(I+A) I/population  
2013-03   1.000000      17.0  2.0  19.0     0.8     0.019699  
2013-04   1.833333       NaN  NaN   NaN     NaN          NaN  
2013-05   2.527778       NaN  NaN   NaN     NaN          NaN  
2013-06  12.106481       NaN  NaN   NaN     NaN          NaN  
2013-07  19.088735       NaN  NaN   NaN     NaN          NaN  
2013-08  33.907279       NaN  NaN   NaN     NaN          NaN  
2013-09  45.256066       NaN  NaN   NaN     NaN          NaN  
2013-10  54.713388       NaN  NaN   NaN     NaN          NaN  
2013-11  47.594490       NaN  NaN   NaN     NaN          NaN  

[9 rows x 98 columns]

In [37]:
df.columns.levels[0]

Index(['0.0', '1.0', '100.0', '101.0', '102.0', '104.0', '92.0', '93.0',
       '94.0', '95.0', '96.0', '97.0', '98.0', '99.0'],
      dtype='object')

In [23]:
df=df.swaplevel(axis=1)

In [27]:
df[['I','A']]

I                                                     \
            104.0     102.0     101.0     100.0       99.0      98.0   
2013-03  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000   
2013-04  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000   
2013-05  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000   
2013-06  0.000000  0.000000  0.000000  0.000000   2.000000  0.000000   
2013-07  0.000000  0.000000  1.000000  0.000000   3.666667  2.000000   
2013-08  2.000000  0.000000  0.833333  2.000000   3.055556  3.666667   
2013-09  1.666667  0.000000  0.694444  1.666667   5.546296  3.055556   
2013-10  1.388889  1.000000  1.578704  1.388889  10.621914  2.546296   
2013-11  1.157407  0.833333  1.315586  1.157407   9.851595  2.121914   

                                                 ...     A                 \
             97.0      96.0      95.0      94.0  ...  99.0 98.0 97.0 96.0   
2013-03  0.000000  0.000000  0.000000  0.000000  ...   3.0  1.0  3.0  1.0   
2013-04  0.000000  0.000000  0.000000  0.000000  ...   NaN  NaN  NaN  NaN   
2013-05  0.000000  1.000000  0.000000  0.000000  ...   NaN  NaN  NaN  NaN   
2013-06  0.000000  0.833333  0.000000  0.000000  ...   NaN  NaN  NaN  NaN   
2013-07  0.000000  0.694444  1.000000  0.000000  ...   NaN  NaN  NaN  NaN   
2013-08  0.000000  0.578704  1.833333  0.000000  ...   NaN  NaN  NaN  NaN   
2013-09  2.000000  1.482253  1.527778  1.000000  ...   NaN  NaN  NaN  NaN   
2013-10  4.666667  3.235211  1.273148  0.833333  ...   NaN  NaN  NaN  NaN   
2013-11  3.888889  2.696009  1.060957  0.694444  ...   NaN  NaN  NaN  NaN   

                                        
        95.0 94.0 93.0 92.0  0.0   1.0  
2013-03  1.0  2.0  2.0  0.0  1.0  19.0  
2013-04  NaN  NaN  NaN  NaN  NaN   NaN  
2013-05  NaN  NaN  NaN  NaN  NaN   NaN  
2013-06  NaN  NaN  NaN  NaN  NaN   NaN  
2013-07  NaN  NaN  NaN  NaN  NaN   NaN  
2013-08  NaN  NaN  NaN  NaN  NaN   NaN  
2013-09  NaN  NaN  NaN  NaN  NaN   NaN  
2013-10  NaN  NaN  NaN  NaN  NaN   NaN  
2013-11  NaN  NaN  NaN  NaN  NaN   NaN  

[9 rows x 28 columns]